# Automated ML

Below are the dependencies that will be needed to complete the project.

In [1]:
import azureml.core
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.dataset import Dataset
from azureml.core.workspace import Workspace
from azureml.core import Experiment, Webservice, Model
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.conda_dependencies import CondaDependencies


import pandas as pd
import sklearn
import json

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.19.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.subscription_id, sep = '\n')

quick-starts-ws-132535
aml-quickstarts-132535
f9d5a085-54dc-4215-9ba6-dad5d86e60a0


In [3]:
# choose a name for experiment
experiment_name = 'capstone-heart-failure-exp'

experiment=Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
capstone-heart-failure-exp,quick-starts-ws-132535,Link to Azure Machine Learning studio,Link to Documentation


In [4]:
# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
cluster_name="cpu-cluster"

try:
    new_cluster=ComputeTarget(workspace=ws, name="cpu-cluster")
    print("Existing cluster detected, make use of it!")
    
except ComputeTargetException:
    print("New compute cluster creation in progress...")
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',vm_priority='dedicated',min_nodes=None,max_nodes=4)
    new_cluster = ComputeTarget.create(ws, "new_cluster", compute_config)    
    new_cluster.wait_for_completion(show_output=True)         
    print("Cluster is ready")

Existing cluster detected, make use of it!


In [51]:
datastore = ws.get_default_datastore()

datastore.upload(src_dir='./', target_path='dataset/')

Uploading an estimated of 15 files
Target already exists. Skipping upload for dataset/automl.ipynb
Target already exists. Skipping upload for dataset/automl.log
Target already exists. Skipping upload for dataset/azureml_automl.log
Target already exists. Skipping upload for dataset/score.py
Target already exists. Skipping upload for dataset/.ipynb_aml_checkpoints/automl-checkpoint2020-11-30-19-47-52.ipynb
Target already exists. Skipping upload for dataset/.ipynb_checkpoints/automl-checkpoint.ipynb
Target already exists. Skipping upload for dataset/deploy-to-cloud/model-register-and-deploy.ipynb
Target already exists. Skipping upload for dataset/deploy-to-cloud/model-register-and-deploy.yml
Target already exists. Skipping upload for dataset/deploy-to-cloud/README.md
Target already exists. Skipping upload for dataset/deploy-to-cloud/score.py
Target already exists. Skipping upload for dataset/enable-app-insights-in-production-service/enable-app-insights-in-production-service.ipynb
Target a

$AZUREML_DATAREFERENCE_5634db4b0fe74072b91bcb5495598737

In [6]:
dataset = Dataset.get_by_name(ws, name="heart-failure")
print("Dataset is registered")
dataset

Dataset is registered


{
  "source": [
    "('workspaceblobstore', 'UI/12-30-2020_075053_UTC/heart_failure_clinical_records_dataset.csv')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ],
  "registration": {
    "id": "ca7c8ad6-58e5-4b4b-ad00-6b116befbde0",
    "name": "heart-failure",
    "version": 1,
    "description": "External data from kaggle",
    "workspace": "Workspace.create(name='quick-starts-ws-132535', subscription_id='f9d5a085-54dc-4215-9ba6-dad5d86e60a0', resource_group='aml-quickstarts-132535')"
  }
}

In [7]:
dataset.take(5).to_pandas_dataframe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


## AutoML Configuration

The following code shows a basic example of creating an AutoMLConfig object and submitting an experiment for classification.
I chose the automl settings below because I wanted to specify the experiment type as classification. The classification experiment will be carried out using AUC weighted as the primary metric with experiment timeout minutes set to 30 minutes and 5 cross-validation folds with the maximum number of iterations that would be executed concurrently set to 4.
All of these settings defines the machine learning task.


The configuration object below contains and persists the parameters for configuring the experiment run, as well as the training data to be used at run time.

In [8]:
automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted',
    "n_cross_validations": 5
}
automl_config = AutoMLConfig(compute_target=new_cluster,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT", 
                             enable_early_stopping= True,
                             featurization= 'auto',
                             enable_voting_ensemble= True,
                             **automl_settings
                            )

In [9]:
# TODO: Submit your experiment

remote_run = experiment.submit(automl_config, show_output=True)

Running on remote.
No run_configuration provided, running on cpu-cluster with default configuration
Running on remote compute: cpu-cluster
Parent Run ID: AutoML_61692633-c857-46d4-a6b3-629dfff50e12

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: 

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [11]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [12]:
remote_run.wait_for_completion(show_output=True)



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more abo

{'runId': 'AutoML_61692633-c857-46d4-a6b3-629dfff50e12',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2020-12-30T19:53:00.152367Z',
 'endTimeUtc': '2020-12-30T20:18:39.441908Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'cpu-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"capstone-heart-failure-exp","subscription_id":"f9d5a085-54dc-4215-9ba6-dad5d86e60a0","resource_group":"aml-quickstarts-132535","workspace_name":"quick-starts-ws-132535","region":"southcentralus","compute_target":"cpu-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"AUC_weighted","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":null,

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [60]:
best_run_id, fitted_model = remote_run.get_output()
print(best_run_id)
print(fitted_model)

best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name)
    print(metric)

Run(Experiment: capstone-heart-failure-exp,
Id: AutoML_61692633-c857-46d4-a6b3-629dfff50e12_51,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                    n_estimators=50,
                                                                                                    n_jobs=1,
                         

In [15]:
#TODO: Save the best model

best_run.register_model(model_name='automl_model',model_path='/outputs',properties={'AUC_weighted':best_run_metrics['AUC_weighted']},tags={'Training context':'Auto ML'})

Model(workspace=Workspace.create(name='quick-starts-ws-132535', subscription_id='f9d5a085-54dc-4215-9ba6-dad5d86e60a0', resource_group='aml-quickstarts-132535'), name=automl_model, id=automl_model:1, version=1, tags={'Training context': 'Auto ML'}, properties={'AUC_weighted': '0.9207544066076043'})

In [16]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
capstone-heart-failure-exp,AutoML_61692633-c857-46d4-a6b3-629dfff50e12_51,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [55]:
model_name = best_run.properties['model_name']
model_name

'AutoML61692633c51'

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [56]:
# Register the model

model = remote_run.register_model(model_name = model_name,
                                  description = 'Automl model')

In [57]:
environment = Environment('capstone-project-environment')
environment.python.conda_dependencies = CondaDependencies.create(pip_packages=[
    'azureml-defaults',
    'inference-schema[numpy-support]',
    'joblib',
    'numpy',
    'scikit-learn=={}'.format(sklearn.__version__)
])

In [62]:
with open('score.py') as f:
    print(f.read())

import os
import pickle
import json
import numpy
from sklearn.externals import joblib

def init():
    global model
   # AZUREML_MODEL_DIR is an environment variable created during deployment.
    # It is the path to the model folder (./azureml-models/$MODEL_NAME/$VERSION)
    # For multiple models, it points to the folder containing all deployed models (./azureml-models)
    # model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'sklearn_classification_model.pkl')

    # deserialize the model file back into a sklearn model
    model = joblib.load(model_path)


# note you can pass in multiple rows for scoring
def run(raw_data):
    try:
        data = json.loads(raw_data)['data']
        result = model.predict(data)
        # you can return any datatype as long as it is JSON-serializable
        return result.tolist()
    except Exception as e:
        error = str(e)
        return error



In [63]:
# Create an inference config
service_name='capstone-service'

inference_config = InferenceConfig(entry_script='score.py', environment=environment)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, description="Predict death event using classification model", enable_app_insights=True)

service = Model.deploy(workspace=ws,
                       name='capstone-encrypted-deploy',
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config
                       )

In [ ]:
service.wait_for_deployment(show_output=True)
print("State: " + service.state)

In [ ]:
aci_service_name = "aci-service-appinsights"

aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aci_deployment_config, overwrite=True)
aci_service.wait_for_deployment(show_output=True)

print(aci_service.state)

In [ ]:
if aci_service.state == "Healthy":
    test_sample = json.dumps({
        "data": [
            [1,28,13,45,54,6,57,8,8,10],
            [101,9,8,37,6,45,4,3,2,41]
        ]
    })

    prediction = aci_service.run(test_sample)

    print(prediction)
else:
    raise ValueError("Service deployment isn't healthy, can't call the service. Error: ", aci_service.error)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
input_payload = json.dumps({
    'data': dataset_x[0:2].tolist()
})

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
# Print the logs of the web service

output = service.run(input_payload)

print(output)

# Delete the service
service.delete()